In [1]:
%reload_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
pd.options.display.max_columns, pd.options.display.max_rows = 500, 200
pd.options.display.float_format = '{:.1f}'.format
import warnings
warnings.filterwarnings('ignore')
import joblib

from euro_soccer.preprocessing import DataPreprocessor
from euro_soccer.glicko_soccer import GlickoSoccer

In [2]:
matches = pd.read_csv('matches.csv')
matches = DataPreprocessor().preprocessing(matches)
matches.tail(1)

,index,date,home_team,away_team,country,tournament,season,tournament_type,outcome,is_pandemic,draw_probability
130638,309703,2022-12-31,Hapoel Afula,H. Akko,Israel,Israel. Leumit League,2021,2,D,0,0.3


In [4]:
count_matches, count_seasons, count_tournaments = DataPreprocessor().test_data(matches)

In [8]:
# count_matches

In [9]:
# count_seasons

In [10]:
# count_tournaments

In [11]:
matches.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 115657 entries, 0 to 130638
Data columns (total 11 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   index             115657 non-null  int64         
 1   date              115657 non-null  datetime64[ns]
 2   home_team         115657 non-null  object        
 3   away_team         115657 non-null  object        
 4   country           115657 non-null  object        
 5   tournament        115657 non-null  object        
 6   season            115657 non-null  int64         
 7   tournament_type   115657 non-null  int64         
 8   outcome           115657 non-null  object        
 9   is_pandemic       115657 non-null  int64         
 10  draw_probability  115657 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(4), object(5)
memory usage: 10.6+ MB


In [22]:
# matches['draw_probability'].mean()

In [23]:
# # matches.loc[(matches['home_team'].str.contains('B')) & (matches['away_team'].str.contains('AS Roma'))]

In [24]:
# matches.info()

In [25]:
# matches['season'].value_counts()

In [26]:
# matches.loc[matches['tournament'].str.contains('Con')]

In [27]:
# matches.loc[matches['country'].str.contains('Uz')]

# Fit Params

In [ ]:
glicko = GlickoSoccer()
league_params = glicko.fit_params(matches, 50, is_params_initialization=False)

Current Loss: 39105.23423495512
Wales. Cymru Premier Loss down by: 0.19
{'init_mu': 1310, 'init_rd': 140, 'update_rd': 34, 'lift_update_mu': 20, 'home_advantage': 28, 'pandemic_home_advantage': 19, 'new_team_update_mu': 0}
England. Championship Loss down by: 0.29
{'init_mu': 1530, 'init_rd': 140, 'update_rd': 34, 'lift_update_mu': -20, 'home_advantage': 28, 'pandemic_home_advantage': 19, 'new_team_update_mu': -102}
Israel. Leumit League Loss down by: 0.13
{'init_mu': 1390, 'init_rd': 140, 'update_rd': 34, 'lift_update_mu': -80, 'home_advantage': 28, 'pandemic_home_advantage': 19, 'new_team_update_mu': -22}
England. Premier League Loss down by: 0.17
{'init_mu': 1690, 'init_rd': 140, 'update_rd': 34, 'lift_update_mu': 80, 'home_advantage': 28, 'pandemic_home_advantage': 19, 'new_team_update_mu': 0}
Gibraltar. National League Loss down by: 0.87
{'init_mu': 1380, 'init_rd': 140, 'update_rd': 34, 'lift_update_mu': 20, 'home_advantage': 28, 'pandemic_home_advantage': 19, 'new_team_update_mu'

# Ratings

In [12]:
glicko = GlickoSoccer()
# league_params = joblib.load('data/league_params.pkl')
league_params = glicko._league_params_initialization(matches)
ratings = glicko.rate_teams(matches, league_params)
ratings = glicko.ratings_to_df(ratings, matches)
leagues_ratings = glicko.league_ratings(ratings, number_top_teams=10)
first_leagues = set(matches.loc[matches['tournament_type'] == 1, 'tournament'])
leagues_ratings = leagues_ratings.loc[leagues_ratings['league'].isin(first_leagues)]
leagues_ratings.head(60)

,#,league,rating
0,1,England. Premier League,1820.3
1,2,Italy. Serie A,1776.2
2,3,Spain. LaLiga,1774.9
3,4,Germany. Bundesliga,1744.1
4,5,Netherlands. Eredivisie,1736.4
5,6,France. Ligue 1,1733.2
6,7,Cyprus. First Division,1727.8
7,8,Norway. Eliteserien,1710.4
8,9,Czech Republic. 1. Liga,1707.7
9,10,Portugal. Primeira Liga,1706.4


In [6]:
leagues_ratings.head(60)

,#,league,rating
0,1,England. Premier League,1834.3
1,2,Italy. Serie A,1799.0
2,3,Spain. LaLiga,1793.3
3,4,Germany. Bundesliga,1756.7
4,5,Cyprus. First Division,1742.2
5,6,France. Ligue 1,1737.8
6,7,Netherlands. Eredivisie,1730.5
7,8,Portugal. Primeira Liga,1722.2
8,9,Belarus. Vysshaya Liga,1709.9
9,10,Serbia. Super Liga,1709.4


In [13]:
ratings.head(50)

,#,team,rating,league
0,1,Liverpool,2005.1,England. Premier League
1,2,Bayern Munich,1996.6,Germany. Bundesliga
2,3,Crvena zvezda,1986.9,Serbia. Super Liga
3,4,Manchester City,1979.8,England. Premier League
4,5,FC Porto,1958.9,Portugal. Primeira Liga
5,6,Real Madrid,1954.9,Spain. LaLiga
6,7,Ajax,1949.3,Netherlands. Eredivisie
7,8,Rangers,1947.9,Scotland. Premiership
8,9,Sheriff Tiraspol,1923.7,Moldova. Divizia Nationala
9,10,Inter,1916.5,Italy. Serie A


# Skellam

In [9]:
# from euro_soccer.draw_model import DrawLightGBM

DrawLightGBM().cross_val_score()

OSError: dlopen(/Users/andreyshelopugin/opt/anaconda3/lib/python3.8/site-packages/lightgbm/lib_lightgbm.so, 6): Library not loaded: /usr/local/opt/libomp/lib/libomp.dylib
  Referenced from: /Users/andreyshelopugin/opt/anaconda3/lib/python3.8/site-packages/lightgbm/lib_lightgbm.so
  Reason: image not found

In [5]:
# import joblib
train = joblib.load('data/train.pkl')
train.head()

,team,score,index,is_home,avg_scoring_10,avg_scoring_20,is_pandemic,mean_score_10,median_score_10,mean_score_20,mean_score_10_against,median_score_10_against,mean_score_20_against
129670,Den Bosch,2,69077,0,2.500000,2.5,1,1.800000,1.5,1.5,1.700000,1.0,2.0
4917,Dubnica,5,67608,1,1.444444,NaN,0,0.777778,1.0,1.0,3.222222,3.0,3.0
17732,Rapid Vienna,2,27656,1,2.400000,2.4,0,1.100000,0.5,1.0,0.600000,0.0,1.0
132205,Chambly,2,51277,0,1.200000,1.2,1,1.100000,0.5,1.0,0.800000,0.5,1.0
115031,Tobol,0,10064,0,1.300000,1.3,0,1.200000,1.0,1.0,0.300000,0.0,0.0


In [6]:
# train.loc[train['team'] == 'Manchester City']

,team,score,index,is_home,avg_scoring_10,avg_scoring_20,is_pandemic,mean_score_10,median_score_10,mean_score_20,mean_score_10_against,median_score_10_against,mean_score_20_against
99593,Manchester City,2,8107,0,2.8,2.8,0,1.700000,1.5,2.0,1.000000,1.0,0.5
97208,Manchester City,1,75728,0,2.7,2.7,0,1.900000,2.0,2.0,1.300000,1.5,1.0
101412,Manchester City,3,8079,0,2.6,2.6,0,1.700000,1.5,2.0,0.800000,0.5,0.5
22198,Manchester City,1,8075,1,1.5,1.5,0,3.300000,3.0,3.0,0.900000,0.5,1.0
112351,Manchester City,3,89736,0,1.7,1.7,0,2.500000,2.5,2.0,0.700000,0.5,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
139024,Manchester City,4,82632,0,1.4,1.4,1,1.100000,1.0,1.0,0.400000,0.0,0.5
51661,Manchester City,2,82643,1,3.1,3.1,1,2.900000,2.5,2.0,0.600000,0.0,1.0
89840,Manchester City,3,80587,0,2.6,2.6,0,1.666667,1.0,1.0,1.777778,1.0,2.0
16367,Manchester City,3,8163,1,2.2,2.2,0,3.200000,3.0,3.0,0.900000,1.0,1.0


In [7]:
# train['avg_scoring_10'].mean()

1.364716147298895